In [89]:
%%time
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , precision_recall_curve , f1_score
from sklearn.metrics import classification_report , roc_auc_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from joblib import dump, load
import joblib

Wall time: 153 ms


In [90]:
rfc = joblib.load('modelmmww.joblib')

In [91]:
df = pd.read_csv('RosterLinkedIn_YaleW2.csv')
#df2 = pd.read_csv('RosterLinkedIn_CornellW.csv')

In [92]:
#df['SameTennis_yn'] = (df['SameName_yn'] + df['tennis_yn'])
#df2['SameTennis_yn'] = (df2['SameName_yn'] + df2['tennis_yn'])


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 33 columns):
FirstLast         259 non-null object
Criteria          259 non-null object
Year              259 non-null int64
YearsPlayed       259 non-null int64
RosterLink        40 non-null object
Hometown          40 non-null object
HS                40 non-null object
ClassYear         40 non-null object
College           259 non-null object
First             259 non-null object
Gender            259 non-null int64
Last              259 non-null object
MidInit           106 non-null object
MidName           1 non-null object
Name              40 non-null object
RosterYear        40 non-null object
LName             202 non-null object
SearchResult      202 non-null object
Headline          202 non-null object
Location          202 non-null object
Company           193 non-null object
School            202 non-null object
Activites         202 non-null object
fuzz_ratio        202 non-null f

In [94]:
#This was for CornellW_merge which had names with no result, e.g. Nan rows
df.dropna(subset = ['SearchResult'], inplace=True)
df.set_index('FirstLast', inplace=True, drop=False)
df = df.select_dtypes(exclude=['object'])
#df_label = df[['EC_Review']]
#df.drop(['EC_Review', 'prediction', 'Decade', 'YearsPlayed'], axis=1, inplace=True)

In [95]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202 entries, Sheila Ford to Kathy Wang
Data columns (total 13 columns):
Year              202 non-null int64
YearsPlayed       202 non-null int64
Gender            202 non-null int64
fuzz_ratio        202 non-null float64
fuzz_pratio       202 non-null float64
fuzz_tokensort    202 non-null float64
fuzz_tokenset     202 non-null float64
fuzz_wratio       202 non-null float64
tennis_yn         202 non-null float64
School_yn         202 non-null float64
SameName_yn       202 non-null float64
SameTennis_yn     202 non-null float64
Decade            202 non-null int64
dtypes: float64(9), int64(4)
memory usage: 22.1+ KB


In [97]:
df

,Year,YearsPlayed,Gender,fuzz_ratio,fuzz_pratio,fuzz_tokensort,fuzz_tokenset,fuzz_wratio,tennis_yn,School_yn,SameName_yn,SameTennis_yn,Decade
FirstLast,,,,,,,,,,,,,
Sheila Ford,1973,1,0,31.00,18.00,23.00,23.00,31.00,0.00,0.00,0.00,0.00,1970
Sarah Fox,1973,1,0,100.00,100.00,100.00,100.00,100.00,0.00,0.00,1.00,1.00,1970
Diane Straus,1973,2,0,96.00,100.00,96.00,96.00,96.00,0.00,0.00,0.00,0.00,1970
Lucy Daggett,1974,2,0,100.00,100.00,100.00,100.00,100.00,0.00,1.00,1.00,1.00,1970
Margaret Mercer,1974,3,0,71.00,67.00,71.00,71.00,71.00,0.00,1.00,0.00,0.00,1970
Deborah Rhode,1974,1,0,67.00,73.00,67.00,78.00,74.00,0.00,1.00,0.00,0.00,1970
Deborah Ritter,1974,1,0,64.00,73.00,64.00,78.00,74.00,0.00,0.00,0.00,0.00,1970
Polly Smith,1974,1,0,55.00,55.00,55.00,62.00,59.00,0.00,0.00,0.00,0.00,1970
Carol Williams,1975,1,0,59.00,62.00,59.00,59.00,59.00,0.00,0.00,0.00,0.00,1970


In [98]:
X = df.values

In [99]:
X.shape

(202, 13)

In [100]:
rfc

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rfc',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                  

In [101]:
rfcpredictX = rfc.predict(X)

In [102]:
rfcpredictX

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.])

In [103]:
df['prediction'] = rfcpredictX

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202 entries, Sheila Ford to Kathy Wang
Data columns (total 14 columns):
Year              202 non-null int64
YearsPlayed       202 non-null int64
Gender            202 non-null int64
fuzz_ratio        202 non-null float64
fuzz_pratio       202 non-null float64
fuzz_tokensort    202 non-null float64
fuzz_tokenset     202 non-null float64
fuzz_wratio       202 non-null float64
tennis_yn         202 non-null float64
School_yn         202 non-null float64
SameName_yn       202 non-null float64
SameTennis_yn     202 non-null float64
Decade            202 non-null int64
prediction        202 non-null float64
dtypes: float64(10), int64(4)
memory usage: 23.7+ KB


In [105]:
df.to_csv('Predict_YaleW2_mmww.csv', index=True)